# Jet types at the LHC

Jets are reconstructed physics objects representing the hadronization and fragmentation of quarks and gluons. CMS mostly uses anti-$k_{\mathrm{T}}$ jets with a cone-size of $R=0.4$ to reconstruct this type of jet. We have algorithms that distinguish heavy-flavour (b or c) quarks (which are in the domain of the BTV POG), quark- vs gluon-originated jets, and jets from the main $pp$ collision versus jets formed largely from pileup particles. 

However, quarks and gluons are only part of the story! At the LHC, the typical collision energy is much greater than the mass scale of the known SM particles, and hence even heavier particles like top quarks, W/Z/Higgs bosons, and heavy beyond-the-Standard-Model particles can be produced with large Lorentz boosts. When these particles decay to quarks and gluons, their decay products are collimated and overlap in the detector, making them difficult to reconstruct as individual AK4 jets. 

Therefore, LHC analyses use jet algorithms with a large radius parameter to reconstruct these objects, which we called "large radius" or "fat" jets. CMS uses anti-$k_{\mathrm{T}}$ jets with $R=0.8$ (AK8) as the standard large-radius jet, while ATLAS uses AK10. 

This topic was explained in more detailed in the slides [ADD LINK TO SLIDES]. You can also read these excellent overviews of jet substructure techniques:

- [Boosted objects: a probe of beyond the Standard Model physics](http://arxiv.org/abs/1012.5412) by Abdesselam et al.
- [Looking inside jets: an introduction to jet substructure and boosted-object phenomenology](https://arxiv.org/abs/1901.10342) by Marzani, Soyez, and Spannowsky.

## Jet types and algorithms in CMS

The standard jet algorithms are all implemented in the CMS reconstruction software, [CMSSW](github.com/cms-sw/cmssw). However, a few algorithms with specific parameters (namely AK4, AK8, and CA15) have become standard tools in CMS; these jet types are extensively studied by the JetMET POG, and are highly recommended. These algorithms are included in the centrally produced CMS samples, at the AOD, miniAOD, and nanoAOD data tiers (note that miniAOD and nanoAOD are most commonly used for analysis, while AOD is much less common these days, and is not widely available on the grid). Other algorithms can be implemented and tested using the **JetToolbox** (discussed later in the tutorial).  

In this part of the tutorial, you will learn how to access the jet collection included in the CMS datasets, do some comparisons of the different jet types, and how to create your own collections. 


### AOD 

[This twiki](https://twiki.cern.ch/twiki/bin/view/CMSPublic/SWGuideDataFormatRecoJets) summarizes the respective labels by which each jet collection can be retrieved from the event record for general AOD files. This format is currently been used for specialized studies, but for most of the analyses you can use the other formats.

### MiniAOD

There are three main jet collections stored in the MiniAOD format, as described [here](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookMiniAOD2017#Jets).
 * **slimmedJets**: are AK4 energy-corrected jets using charged hadron substraction (CHS) as the pileup removal algorithm. This is the default jet collection for CMS analyses for Run II. In this collection you can find the following jet algorithms, as well as other jet related quantities:
   * b-tagging 
   * Pileup jet ID
   * Quark/gluon likelihood info embedded.
 * **slimmedJetsPUPPI**: are AK4 energy-corrected jets using the PUPPI algorithm for pileup removal. This collection will be the default for Run III analyses.
 * **slimmedJetsAK8**: ak4 AK8 energy-corrected jets using the PUPPI algoritm for pileup removal. This has been the default collection for boosted jets in Run II. In this collection you can find the following jet algorithms, as well as other jet related quantities:
   * Softdrop mass
   * n-subjettiness and energy correlation variables
   * Access to softdrop subjets
   * Access to the associated AK8 CHS jet four momentum, including softdrop and pruned mass, and n-subjettiness.

#### Examples of how to access jet collections in miniAOD samples

Below are two examples of how to access jet collections from these samples. This exercise does not intend for you to modify code in order to access these collections, but rather for you to look at the code and get an idea about how you could access this information if needed.

##### In C++

If you are on the terminal, please take a look at the file `$CMSSW_BASE/src/Analysis/JMEDAS/src/jmedas_miniAODAnalyzer.C`.  The same code can be seen by clicking in the link below:

<details>
    <summary><font color='blue'>SHOW CODE</font></summary>
<p>

In C++ you need to create a .C file inside the `src` directory. 
    
```C
// system include files
#include <memory>
#include <vector>

// user include files
#include "FWCore/Framework/interface/Frameworkfwd.h"
#include "FWCore/Framework/interface/EDAnalyzer.h"

#include "FWCore/Framework/interface/Event.h"
#include "FWCore/Framework/interface/MakerMacros.h"

#include "FWCore/ParameterSet/interface/ParameterSet.h"

#include "DataFormats/VertexReco/interface/VertexFwd.h"
#include "DataFormats/VertexReco/interface/Vertex.h"
#include "DataFormats/PatCandidates/interface/Jet.h"
#include "DataFormats/PatCandidates/interface/PackedCandidate.h"
//
// class declaration
//
using namespace std;

class MiniAnalyzer : public edm::EDAnalyzer {
   public:
      explicit MiniAnalyzer (const edm::ParameterSet&);
      ~MiniAnalyzer();

   private:
      virtual void analyze(const edm::Event&, const edm::EventSetup&) override;

      // ----------member data ---------------------------
      edm::EDGetTokenT<vector<reco::Vertex>> vtxToken_;
      edm::EDGetTokenT<pat::JetCollection> jetToken_;
      edm::EDGetTokenT<pat::JetCollection> fatjetToken_;

};

MiniAnalyzer::MiniAnalyzer(const edm::ParameterSet& iConfig):
    vtxToken_(consumes<vector<reco::Vertex>>(iConfig.getParameter<edm::InputTag>("vertices"))),
    jetToken_(consumes<pat::JetCollection>(iConfig.getParameter<edm::InputTag>("jets"))),
    fatjetToken_(consumes<pat::JetCollection>(iConfig.getParameter<edm::InputTag>("fatjets")))
{
}

MiniAnalyzer::~MiniAnalyzer()
{
}

void
MiniAnalyzer::analyze(const edm::Event& iEvent, const edm::EventSetup& iSetup)
{

    edm::Handle<vector<reco::Vertex>> vertices;
    iEvent.getByToken(vtxToken_, vertices);
    if (vertices->empty()) return; // skip the event if no PV found
    const reco::Vertex &PV = vertices->front();

    edm::Handle<pat::JetCollection> jets;
    iEvent.getByToken(jetToken_, jets);
    int ijet = 0;
    for (const pat::Jet &j : *jets) {
        if (j.pt() < 20) continue;
        printf("jet  with pt %5.1f (raw pt %5.1f), eta %+4.2f, btag CSV %.3f, CISV %.3f, pileup mva disc %+.2f\n",
            j.pt(), j.pt()*j.jecFactor("Uncorrected"), j.eta(), std::max(0.f,j.bDiscriminator("combinedSecondaryVertexBJetTags")), std::max(0.f,j.bDiscriminator("combinedInclusiveSecondaryVertexBJetTags")), j.userFloat("pileupJetId:fullDiscriminant"));
        if ((++ijet) == 1) { // for the first jet, lets print the leading constituents
            std::vector daus(j.daughterPtrVector());
            std::sort(daus.begin(), daus.end(), [](const reco::CandidatePtr &p1, const reco::CandidatePtr &p2) { return p1->pt() > p2->pt(); }); // the joys of C++11
            for (unsigned int i2 = 0, n = daus.size(); i2 < n && i2 <= 3; ++i2) {
                const pat::PackedCandidate &cand = dynamic_cast<const pat::PackedCandidate &>(*daus[i2]);
                printf("         constituent %3d: pt %6.2f, dz(pv) %+.3f, pdgId %+3d\n", i2,cand.pt(),cand.dz(PV.position()),cand.pdgId());
            }
        }
    }

    edm::Handle<pat::JetCollection> fatjets;
    iEvent.getByToken(fatjetToken_, fatjets);
    for (const pat::Jet &j : *fatjets) {
        printf("AK8j with pt %5.1f (raw pt %5.1f), eta %+4.2f, mass %5.1f ungroomed, %5.1f softdrop, %5.1f pruned CHS\n",
            j.pt(), j.pt()*j.jecFactor("Uncorrected"), j.eta(), j.mass(), j.userFloat("ak8PFJetsPuppiSoftDropMass"), j.userFloat("ak8PFJetsCHSValueMap:ak8PFJetsCHSPrunedMass") );

        // To get the constituents of the AK8 jets, you have to loop over all of the
        // daughters recursively. To save space, the first two constituents are actually
        // the Soft Drop SUBJETS, which will then point to their daughters.
        // The remaining constituents are those constituents removed by soft drop but
        // still in the AK8 jet.
        std::vector<reco::Candidate const *> constituents;
        for ( unsigned ida = 0; ida < j.numberOfDaughters(); ++ida ) {
     reco::Candidate const * cand = j.daughter(ida);
     if ( cand->numberOfDaughters() == 0 )
       constituents.push_back( cand ) ;
     else {
       for ( unsigned jda = 0; jda < cand->numberOfDaughters(); ++jda ) {
         reco::Candidate const * cand2 = cand->daughter(jda);
         constituents.push_back( cand2 );
       }
     }
   }
   std::sort( constituents.begin(), constituents.end(), [] (reco::Candidate const * ida, reco::Candidate const * jda){return ida->pt() > jda->pt();} );

   for ( unsigned int ida = 0; ida < constituents.size(); ++ida ) {
     const pat::PackedCandidate &cand = dynamic_cast<const pat::PackedCandidate &>(*constituents[ida]);
     printf("         constituent %3d: pt %6.2f, dz(pv) %+.3f, pdgId %+3d\n", ida,cand.pt(),cand.dz(PV.position()),cand.pdgId());
   }

   auto sdSubjets = j.subjets("SoftDropPuppi");
   for ( auto const & isd : sdSubjets ) {
     printf("  sd subjet with pt %5.1f (raw pt %5.1f), eta %+4.2f, mass %5.1f ungroomed\n",
       isd->pt(), isd->pt()*isd->jecFactor("Uncorrected"), isd->eta(), isd->mass() );

   }
    }
}


//define this as a plug-in
DEFINE_FWK_MODULE(MiniAnalyzer);
```

Then, you run your C script with a python configuration, through the cmsRun command:

```python
import FWCore.ParameterSet.Config as cms

process = cms.Process("Demo")

process.load("FWCore.MessageService.MessageLogger_cfi")
process.maxEvents = cms.untracked.PSet( input = cms.untracked.int32(10)  )

process.source = cms.Source("PoolSource",
                            fileNames = cms.untracked.vstring(
                                        '/store/mc/RunIIAutumn18MiniAOD/QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/102X_upgrade2018_realistic_v15-v1/210000/DA20DC21-E781-C540-9FCD-7BCF2144CA4E.root'

                            )
                            )

process.demo = cms.EDAnalyzer("MiniAnalyzer",
                                  vertices = cms.InputTag("offlineSlimmedPrimaryVertices"),
                                  jets = cms.InputTag("slimmedJets"),
                                  fatjets = cms.InputTag("slimmedJetsAK8")
                              )

process.p = cms.Path(process.demo)
```
            
</p>
</details>

You can run this code by using the python config file `$CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_miniAODtest.py` from your terminal or in the next cell in this notebook. This script will only print out some information about the jets in that sample. Again, the most important part of this exercise is to get familiar with how to access jet collections from miniAOD.

In [ ]:
### RUN THIS CELL ONLY IF YOU ARE USING SWAN 
import os

##### REMEMBER TO MANUALLY COPY THE PROXY TO YOUR CERNBOX FOLDER AND TO MODIFY THE NEXT LINE
os.environ['X509_USER_PROXY'] = '{}/x509up_u5882'.format(os.environ["HOME"])
if os.path.isfile(os.environ['X509_USER_PROXY']): 
    print("Found proxy at {}".format(os.environ['X509_USER_PROXY']))
else: 
    print("Failed to find proxy at {}".format(os.environ['X509_USER_PROXY']))
os.environ['X509_CERT_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/certificates'
os.environ['X509_VOMS_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/vomsdir'

In [ ]:
%%bash
cmsRun $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_miniAODtest.py

##### In Python

If you are on the terminal, please take a look at the file `$CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_miniAODtest_purePython.py`.  The same code can be seen by clicking in the link below:

<details>
    <summary><font color='blue'>SHOW CODE</font></summary>
<p>

```python
# import ROOT in batch mode
import sys
oldargv = sys.argv[:]
sys.argv = [ '-b-' ]
import ROOT
ROOT.gROOT.SetBatch(True)
sys.argv = oldargv

# load FWLite C++ libraries
ROOT.gSystem.Load("libFWCoreFWLite.so");
ROOT.gSystem.Load("libDataFormatsFWLite.so");
ROOT.FWLiteEnabler.enable()

# load FWlite python libraries
from DataFormats.FWLite import Handle, Events

jets, jetLabel = Handle("std::vector<pat::Jet>"), "slimmedJets"
fatjets, fatjetLabel = Handle("std::vector<pat::Jet>"), "slimmedJetsAK8"
vertices, vertexLabel = Handle("std::vector<reco::Vertex>"), "offlineSlimmedPrimaryVertices"
verticesScore = Handle("edm::ValueMap<float>")
seenIt = {} # list of things we've seen (so that we dump them in full only once)

# open file (you can use 'edmFileUtil -d /store/whatever.root' to get the physical file name)
events = Events('root://cms-xrd-global.cern.ch//store/mc/RunIIAutumn18MiniAOD/QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/102X_upgrade2018_realistic_v15-v1/210000/DA20DC21-E781-C540-9FCD-7BCF2144CA4E.root' )

for iev,event in enumerate(events):
    if iev >= 10: break
    event.getByLabel(jetLabel, jets)
    event.getByLabel(fatjetLabel, fatjets)
    event.getByLabel(vertexLabel, vertices)
    event.getByLabel(vertexLabel, verticesScore)

    print "\nEvent %d: run %6d, lumi %4d, event %12d" % (iev,event.eventAuxiliary().run(), event.eventAuxiliary().luminosityBlock(),event.eventAuxiliary().event())

    # Vertices
    if len(vertices.product()) == 0 or vertices.product()[0].ndof() < 4:
        print "Event has no good primary vertex."
        continue
    else:
        PV = vertices.product()[0]
        print "PV at x,y,z = %+5.3f, %+5.3f, %+6.3f, ndof: %.1f, score: (pt2 of clustered objects) %.1f" % (PV.x(), PV.y(), PV.z(), PV.ndof(),verticesScore.product().get(0))


    # Jets (AK4, CHS and Puppi)
    for jetLabel, algo in ("slimmedJets", "CHS"), ("slimmedJetsPuppi", "PUPPI"):
        event.getByLabel(jetLabel, jets)
        for i,j in enumerate(jets.product()):
            if j.pt() < 20: continue
            print "jet %s %3d: pt %5.1f (raw pt %5.1f, matched-calojet pt %5.1f), eta %+4.2f, btag CSVIVFv2 %.3f, CMVAv2 %.3f, pileup mva disc %+.2f" % (
                algo, i, j.pt(), j.pt()*j.jecFactor('Uncorrected'), j.userFloat("caloJetMap:pt") if algo == "CHS" else -99.0, j.eta(), max(0,j.bDiscriminator("pfCombinedInclusiveSecondaryVertexV2BJetTags")), max(0,j.bDiscriminator("pfCombinedMVAV2BJetTags")), j.userFloat("pileupJetId:fullDiscriminant") if algo == "CHS" else -99)
            if 'jetAk4'+algo not in seenIt:
                constituents = [ j.daughter(i2) for i2 in xrange(j.numberOfDaughters()) ]
                constituents.sort(key = lambda c:c.pt(), reverse=True)
                for i2, cand in enumerate(constituents):
                    if i2 > 12:
                            print "         ....."
                            break
                    print "         constituent %3d: pt %6.2f, dz(pv) %+.3f, pdgId %+3d, hcal energy fraction %.2f, puppi weight %.3f " % (i2,cand.pt(),cand.dz(PV.position()),cand.pdgId(),cand.hcalFraction(),cand.puppiWeight())
                print "   btag discriminators:"
                for btag in j.getPairDiscri():
                    print  "\t%s %s" % (btag.first, btag.second)
                print "   userFloats:"
                for ufl in j.userFloatNames():
                    print  "\t%s %s" % (ufl, j.userFloat(ufl))
                seenIt['jetAk4'+algo] = True

    # Fat AK8 Jets
    for i,j in enumerate(fatjets.product()):
        print "jetAK8 %3d: pt %5.1f (raw pt %5.1f), eta %+4.2f, mass %5.1f ungroomed, %5.1f softdrop, %5.1f pruned CHS. " % (
            i, j.pt(), j.pt()*j.jecFactor('Uncorrected'), j.eta(), j.mass(), j.userFloat('ak8PFJetsPuppiSoftDropMass'), j.userFloat('ak8PFJetsCHSValueMap:ak8PFJetsCHSPrunedMass'))
        # If you use CMSSW_9_4_<=6 to read MiniAOD, to get the constituents of the AK8 jets, you have to loop over all of the
        # daughters recursively. To save space, the first two constituents are actually
        # the Soft Drop SUBJETS, which will then point to their daughters.
        # The remaining constituents are those constituents removed by soft drop but
        # still in the AK8 jet.
        if 'jetAk8' not in seenIt:
            constituents = []
            for ida in xrange( j.numberOfDaughters() ) :
                cand = j.daughter(ida)
                if cand.numberOfDaughters() == 0 :
                    constituents.append( cand )
                else : # only needed in CMSSW_9_4_<=6
                    for jda in xrange( cand.numberOfDaughters() ) : # only needed in CMSSW_9_4_<=6
                        cand2 = cand.daughter(jda) # only needed in CMSSW_9_4_<=6
                        constituents.append( cand2 ) # only needed in CMSSW_9_4_<=6
            constituents.sort(key = lambda c:c.pt(), reverse=True)
            for i2, cand in enumerate(constituents):
                if i2 >4:
                            print "         ....."
                            break
                print "         constituent %3d: pt %6.2f, pdgId %+3d, #dau %+3d" % (i2,cand.pt(),cand.pdgId(), cand.numberOfDaughters())
            print "   btag discriminators:"
            for btag in j.getPairDiscri():
                print  "\t%s %s" % (btag.first, btag.second)
            print "   userFloats:"
            for ufl in j.userFloatNames():
                print  "\t%s %s" % (ufl, j.userFloat(ufl))
            seenIt['jetAk8'] = True
        # Print Subjets
        if 'jetAk8SD' not in seenIt:
            sdSubjets = j.subjets('SoftDropPuppi')
            for isd,sdsub in enumerate( sdSubjets ) :
                print "   w subjet %3d: pt %5.1f (raw pt %5.1f), eta %+4.2f, mass %5.1f " % (
                    isd, sdsub.pt(), sdsub.pt()*sdsub.jecFactor('Uncorrected'), sdsub.eta(), sdsub.mass()
                    )
                print "   \tbtag discriminators:"
                for btag in sdsub.getPairDiscri():
                    print  "\t\t%s %s" % (btag.first, btag.second)
                print "   \tuserFloats:"
                for ufl in sdsub.userFloatNames():
                    print  "\t\t%s %s" % (ufl, sdsub.userFloat(ufl))
                seenIt['jetAk8SD'] = True

```
            
</p>
</details>

This code can be run with simple python in your terminal. Similar as in the case for C++, the output of this job is some information about jets. The most important part of the exercise is to get familiar with how to access jet collections using python from miniAOD.

In [ ]:
%%bash
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_miniAODtest_purePython.py

### NanoAOD

In nanoAOD, only AK4 CHS jets ( _Jet_ ) and AK8 PUPPI jets ( _FatJet_ ) are stored. The jets in nanoAOD are similar to those in miniAOD, but not identical (for example, the $p_{\mathrm{T}}$ cuts might be different). A full set of variables for each jet collection can be found in this [website](https://cms-nanoaod-integration.web.cern.ch/integration/master-102X/mc102X_doc.html).

NanoAOD is a "flat tree" format, meaning that you can access the information directly with simple ROOT, or even simple python tools (like numpy or pandas). This format is becoming more and more popular within CMS due to its simplicity and accesibility. An extremely simple example can be found in the next cell (or, if you're using the terminal, `$CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_nanoAODtest.py`). Try running this now.

*Aside*: there are several more advanced tools on the market which allow you to do more sophisticated analysis using nanoAOD format, including [RDataFrame](https://root.cern/doc/master/classROOT_1_1RDataFrame.html), [NanoAOD-tools](https://github.com/cms-nanoAOD/nanoAOD-tools), or [Coffea](https://github.com/CoffeaTeam/coffea). We excourage you to look at them and use the one you like the most.

In [ ]:
from ROOT import *

inputFile = TFile.Open('root://xrootd-cms.infn.it//store/mc/RunIIAutumn18NanoAODv4/QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/Nano14Dec2018_102X_upgrade2018_realistic_v16-v1/110000/C23F77AA-3909-C74E-ADCD-8266FF68AB5D.root' )
events = inputFile.Get('Events')

for iev in xrange(events.GetEntries()):
    events.GetEntry(iev)
    if iev >= 10: break

    print "\nEvent %d: run %6d, lumi %4d, event %12d" % (iev, events.run, events.luminosityBlock, events.event )

    # AK4 CHS Jets
    for ijet in range(events.nJet):
        if events.Jet_pt < 20: continue
        print 'AK4 jet '+ str(ijet) + ': pt ' + str(events.Jet_pt[ijet]) + ', eta ' + str(events.Jet_eta[ijet]) + ', mass ' + str(events.Jet_mass[ijet]) + ',phi ' + str(events.Jet_phi[ijet]) + ', puId ' + str(events.Jet_puId[ijet]) + ', deepJet btag disc. ' + str(events.Jet_btagDeepB[ijet])

    # AK8 PUPPI Jets
    for ijet in range(events.nFatJet):
        if events.FatJet_pt < 20: continue
        print 'AK8 jet '+ str(ijet) + ': pt ' + str(events.FatJet_pt[ijet]) + ', eta ' + str(events.FatJet_eta[ijet]) + ', mass ' + str(events.FatJet_mass[ijet]) + ',phi ' + str(events.FatJet_phi[ijet]) + ', deepAK8 W tag disc. ' + str(events.FatJet_deepTag_WvsQCD[ijet])+ ', deepAK8 top tag disc. ' + str(events.FatJet_deepTag_TvsQCD[ijet])


### jetToolbox

Although JME generally recommends to use AK4 CHS and AK8 PUPPI jets for Run II analyses (moving fully to AK4 PUPPI jets for Run III), there are cases where certain analysis will need to use something else. Similar for the standard algorithms stored in mini/nanoAOD samples. For users who want to test a different jet collection or algorithms, JetMET had developed a user-friendly tool to compute them: [JetToolbox](https://twiki.cern.ch/twiki/bin/view/CMS/JetToolbox).

The JetToolbox is *not* part of CMSSW because JME wants to have the freedom to incorporate and test as many tools as possible without these algorithms being part of any central samples or code. That is the reason that, in real life, you would need to clone the [JetToolbox repository](https://github.com/cms-jet/JetToolbox) inside your CMSSW src folder like this:
~~~
cd $CMSSW_BASE/src/
git clone git@github.com:cms-jet/JetToolbox.git JMEAnalysis/JetToolbox -b jetToolbox_102X_v3
scram b
~~~

In this tutorial, this step was done for you in the initial setup. _You do not need to do it now_. You can find more information about how to set up the JetToolbox in the [README.md](https://github.com/cms-jet/JetToolbox) of the github repository or in the [twiki](https://twiki.cern.ch/twiki/bin/view/CMS/JetToolbox).

For instance, imagine that you want to create a new jet collection using the Cambridge-Aachen clustering algorithm with cone-size 1.2, using PUPPI as pileup removal algorithm, some selection, including the pruning and softdrop grooming algorithms, and use different parameters for the softdrop algoritm. Then in your python config file you need to include something like this:

```
from Analysis.JetToolbox.jetToolbox_cff import *                 # Load the jetTOolbox

jetToolbox( process, 'ca10', 'jetSequence', 'out',               # cone size and basic setup
  Cut="pt>170 && abs(eta)<2.5",                                  # selection
  dataTier="miniAOD",                                            # input file: miniAOD or nanoAOD?
  PUMethod='PUPPI',                                              # PUPPI/CHS/SK/PF
  addPruning=True, addSoftDrop=True , betaCut = 1.0,             # add basic grooming
  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']   # for jet corrections
)
```

A full list of algorithms and parameters can be found in the [JetToolbox twiki](https://twiki.cern.ch/twiki/bin/viewauth/CMS/JetToolbox#Arguments). A more complete example can be found in `$CMSSW_BASE/src/Analysis/JMEDAS/scripts/ClusterWithToolboxAndMakeHistos.py`, or by clicking in the link below.

<details>
    <summary><font color='blue'>SHOW CODE</font></summary>
<p>

```python
import FWCore.ParameterSet.Config as cms
###
### cmsRun ClusterWithToolboxAndPlot.py
###  make jet plots from miniAOD with some additional jet collections clustered
### Jet Algorithm HATS 2015 - Dolen, Rappoccio, Stupak
### Updated for Jet Algorithm HATS 2016 - Dolen, Pilot, Kries, Perloff, Tran
###


process = cms.Process("Ana")

### SETUP
process.load("FWCore.MessageService.MessageLogger_cfi")
process.load("Configuration.EventContent.EventContent_cff")
process.load('Configuration.Geometry.GeometryRecoDB_cff')
process.load('Configuration.StandardSequences.MagneticField_38T_cff')
process.load('Configuration.StandardSequences.Services_cff')
process.load('Configuration.StandardSequences.FrontierConditions_GlobalTag_condDBv2_cff')
process.GlobalTag.globaltag = '92X_upgrade2017_realistic_v1'
process.GlobalTag.globaltag = '94X_mc2017_realistic_v12'
process.options = cms.untracked.PSet( wantSummary = cms.untracked.bool(True) )
process.options.allowUnscheduled = cms.untracked.bool(True)
process.MessageLogger.cerr.FwkReport.reportEvery = 1000

### INPUT
process.maxEvents = cms.untracked.PSet( input = cms.untracked.int32(100) )

process.source = cms.Source("PoolSource",
    fileNames = cms.untracked.vstring('/store/mc/RunIIAutumn18MiniAOD/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/MINIAODSIM/102X_upgrade2018_realistic_v15_ext1-v2/20000/7E65457A-87E5-C146-8321-9A48B4F56ED1.root'),
)

#from PhysicsTools.PatAlgos.patInputFiles_cff import filesRelValTTbarPileUpMINIAODSIM
#process.source.fileNames = filesRelValTTbarPileUpMINIAODSIM

btagDiscAK4 = [
            'pfCombinedInclusiveSecondaryVertexV2BJetTags',
            'pfCombinedCvsLJetTags',
            'pfCombinedCvsBJetTags',
            'pfDeepCSVJetTags:probb',
            'pfDeepCSVJetTags:probbb',
            'pfDeepCSVJetTags:probc',
            'pfDeepCSVJetTags:probudsg',
            'pfDeepFlavourJetTags:probb',
            'pfDeepFlavourJetTags:probbb',
            'pfDeepFlavourJetTags:problepb',
            'pfDeepFlavourJetTags:probc',
            'pfDeepFlavourJetTags:probuds',
            'pfDeepFlavourJetTags:probg',
        ]

btagDiscAK8 = [
            'pfBoostedDoubleSecondaryVertexAK8BJetTags',
            #'pfMassIndependentDeepDoubleBvLJetTags:probQCD',
            #'pfMassIndependentDeepDoubleBvLJetTags:probHbb',
            #'pfMassIndependentDeepDoubleCvLJetTags:probQCD',
            #'pfMassIndependentDeepDoubleCvLJetTags:probHcc',
            #'pfMassIndependentDeepDoubleCvBJetTags:probHbb',
            #'pfMassIndependentDeepDoubleCvBJetTags:probHcc',
	    "pfMassDecorrelatedDeepBoostedDiscriminatorsJetTags:bbvsLight",
	    "pfMassDecorrelatedDeepBoostedDiscriminatorsJetTags:ccvsLight",
	    "pfMassDecorrelatedDeepBoostedDiscriminatorsJetTags:TvsQCD",
	    "pfMassDecorrelatedDeepBoostedDiscriminatorsJetTags:ZHccvsQCD",
 	    "pfMassDecorrelatedDeepBoostedDiscriminatorsJetTags:WvsQCD",
 	    "pfMassDecorrelatedDeepBoostedDiscriminatorsJetTags:ZHbbvsQCD"
            ]

### ADD SOME NEW JET COLLECTIONS
from Analysis.JetToolbox.jetToolbox_cff import *

# updating slimmedJets (ak4 CHS jets) from miniAOD
jetToolbox( process, 'ak4', 'ak4UpdJetSubs', 'noOutput',
        dataTier="miniAOD",
        updateCollection='slimmedJets',
        JETCorrPayload='AK4PFchs',
        postFix='Updated',
        bTagDiscriminators=btagDiscAK4
        )

# AK R=0.4 jets from CHS inputs
jetToolbox( process, 'ak4', 'ak4JetSubs', 'noOutput',
  dataTier="miniAOD",
  PUMethod='CHS',
#  JETCorrPayload = 'AK4PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
  addPUJetID=True,
  addQGTagger=True,
  Cut='pt>30 && abs(eta)<2.5'
)

# AK R=0.4 jets from Puppi inputs
jetToolbox( process, 'ak4', 'ak4JetSubs', 'noOutput',
  dataTier="miniAOD",
  PUMethod='Puppi',
#  JETCorrPayload = 'AK4PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
  Cut='pt>30 && abs(eta)<2.5'
)

# AK R=0.4 jets from PF inputs
jetToolbox( process, 'ak4', 'ak4JetSubs', 'noOutput',
  dataTier="miniAOD",
  PUMethod='Plain',
#  JETCorrPayload = 'AK4PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
  Cut='pt>30 && abs(eta)<2.5'
)



# updating slimmedAK8Jets (ak8 puppi jets) from miniAOD
jetToolbox( process, 'ak8', 'ak8UpdJetSubs', 'noOutput',
        dataTier="miniAOD",
        PUMethod='Puppi',
        updateCollection='slimmedJetsAK8',
        JETCorrPayload='AK8PFPuppi',
        postFix='Updated',
        bTagDiscriminators=btagDiscAK8
        )

# AK R=0.8 jets from CHS
jetToolbox( process, 'ak8', 'ak8JetSubs', 'noOutput',
  dataTier="miniAOD",
  Cut="pt>170 && abs(eta)<2.5",
  PUMethod='CHS',
  addPruning=True, addSoftDrop=True ,           # add basic grooming
  addTrimming=True, addFiltering=True,
  addSoftDropSubjets=True,
  addNsub=True, maxTau=4,                       # add Nsubjettiness tau1, tau2, tau3, tau4
  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute'],
)

# AK R=0.8 from PUPPI
jetToolbox( process, 'ak8', 'ak8JetSubs', 'noOutput',
  dataTier="miniAOD",
  PUMethod='Puppi',
  Cut="pt>170 && abs(eta)<2.5",
  addPruning=True, addSoftDrop=True ,           # add basic grooming
  addTrimming=True, addFiltering=True,
  addSoftDropSubjets=True,
  addNsub=True, maxTau=4,                       # add Nsubjettiness tau1, tau2, tau3, tau4
  addEnergyCorrFunc=True,
  #JETCorrPayload = 'AK8PFPuppi', JETCorrLevels = ['L2Relative', 'L3Absolute']
)

# CA R=0.8 jets from CHS
jetToolbox( process, 'ca8', 'ca8JetSubs', 'noOutput',
  Cut="pt>170 && abs(eta)<2.5",
  dataTier="miniAOD",
  PUMethod='CHS',
  addPruning=True, addSoftDrop=True ,           # add basic grooming
  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
)

# KT R=0.8 jets from CHS
jetToolbox( process, 'kt8', 'kt8JetSubs', 'noOutput',
  Cut="pt>170 && abs(eta)<2.5",
  dataTier="miniAOD",
  PUMethod='CHS',
  addPruning=True, addSoftDrop=True ,           # add basic grooming
  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
)

# KT R=0.8 jets from CHS
jetToolbox( process, 'kt8', 'kt8JetSubs', 'noOutput',
  dataTier="miniAOD",
  Cut="pt>170 && abs(eta)<2.5",
  PUMethod='CHS',
  addPruning=True, addSoftDrop=True ,           # add basic grooming
  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
)

# AK R=0.8 jets from Puppi, modified Softdrop
jetToolbox( process, 'ak8', 'ak8JetSubs', 'noOutput',
  dataTier="miniAOD",
  Cut="pt>170 && abs(eta)<2.5",
  PUMethod='Puppi',
  postFix='modSD',
  addSoftDrop=True , betaCut=1,           # add basic grooming
  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
)

# AK R=0.8 jets from SK
jetToolbox( process, 'ak8', 'ak8JetSubs', 'noOutput',
  dataTier="miniAOD",
  Cut="pt>170 && abs(eta)<2.5",
  PUMethod='SK',
  addPruning=True, addSoftDrop=True ,           # add basic grooming
  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
)

## AK R=0.8 jets from CS
#jetToolbox( process, 'ak8', 'ak8JetSubs', 'noOutput',
#  PUMethod='CS',
#  addPruning=True, addSoftDrop=True ,           # add basic grooming
#  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
#)

# CA R=1.2 jets from CHS inputs with basic grooming, W tagging, and top tagging
#jetToolbox( process, 'ca12', 'ca12JetSubs', 'noOutput',
#  PUMethod='CHS',
#  addHEPTopTagger=True,
#)


## AK R=1.2 jets from CHS inputs with basic grooming, W tagging, and top tagging
#jetToolbox( process, 'ak12', 'ak12JetSubs', 'noOutput',
#  PUMethod='CHS',
#  addPruning=True, addSoftDrop=True ,           # add basic grooming
#  addTrimming=True, addFiltering=True,
#  addSoftDropSubjets=True,
#  addNsub=True, maxTau=4,                       # add Nsubjettiness tau1, tau2, tau3, tau4
#  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
#)
#
## AK R=1.5 jets from CHS inputs with basic grooming, W tagging, and top tagging
#jetToolbox( process, 'ak15', 'ak15JetSubs', 'noOutput',
#  PUMethod='CHS',
#  addPruning=True, addSoftDrop=True ,           # add basic grooming
#  addTrimming=True, addFiltering=True,
#  addSoftDropSubjets=True,
#  addNsub=True, maxTau=4,                       # add Nsubjettiness tau1, tau2, tau3, tau4
#  JETCorrPayload = 'AK8PFchs', JETCorrLevels = ['L2Relative', 'L3Absolute']
#)

### MAKE SOME HISTOGRAMS
process.ana = cms.EDAnalyzer('JetTester'
)

### OUT
process.TFileService = cms.Service("TFileService",
      fileName = cms.string("JetClusterHistos_ZP3.root"),
      closeFileFast = cms.untracked.bool(True)
  )

process.p = cms.Path(process.ana)

```
            
</p>
</details>

Now you can run the script in your terminal, or in the next cell to create a bunch of new jet collections:

In [ ]:
%%bash
cmsRun $CMSSW_BASE/src/Analysis/JMEDAS/scripts/ClusterWithToolboxAndMakeHistos.py

Running the jetToolbox as explained above does create a miniAOD-like output. To know the name of the newly create jet collection, jetToolbox prints out some information like this:
```
|---- jetToolBox: JETTOOLBOX RUNNING ON MiniAOD FOR AK8 JETS USING Puppi
|---- jetToolBox: Applying these corrections: ('AK8PFPuppi', ['L2Relative', 'L3Absolute'], 'None')
|---- jetToolBox: Creating packedPatJetsAK8PFPuppiSoftDrop collection with SoftDrop subjets.
|---- jetToolBox: Running ak8PFJetsPuppiSoftDropMass, selectedPatJetsAK8PFPuppiSoftDropPacked:SubJets, ak8PFJetsPuppiPrunedMass, ak8PFJetsPuppiTrimmedMass, ak8PFJetsPuppiFilteredMass, NjettinessAK8Puppi, nb1AK8PuppiSoftDrop.
|---- jetToolBox: Creating selectedPatJetsAK8PFPuppi collection.
```

The name of the new jet collection in this case will be `selectedPatJetsAK8PFPuppi`. To access it, you can follow the same procedure you did for the `slimmedJets` or `slimmedAK8Jets` in the miniAOD example above.

Because the nanoAOD format is becoming very popular in CMS, there is a "beta" version of the jetToolbox which will create nanoAOD-like samples including new jet collections. For example, to create a nanoAOD like sample, you need to include a file like the one below in your folder `$CMSSW_BASE/src/Analysis/JMEDAS/python/`:

<details>
    <summary><font color='blue'>SHOW CODE</font></summary>
<p>

```python
import FWCore.ParameterSet.Config as cms
from  PhysicsTools.NanoAOD.common_cff import *
### NanoAOD v5 (for 2016,2017,2018), for different recipe please modify accordingly
from Configuration.Eras.Modifier_run2_nanoAOD_94X2016_cff import run2_nanoAOD_94X2016
from Configuration.Eras.Modifier_run2_nanoAOD_94XMiniAODv2_cff import run2_nanoAOD_94XMiniAODv2
from Configuration.Eras.Modifier_run2_nanoAOD_102Xv1_cff import run2_nanoAOD_102Xv1
from JMEAnalysis.JetToolbox.jetToolbox_cff import jetToolbox

# ---------------------------------------------------------
# This is the part the user should modify
def setupCustomizedJetToolbox(process):

    #### AK4 PUPPI jets

    ak4btagdiscriminators = [
            'pfDeepCSVJetTags:probb',
            'pfDeepCSVJetTags:probbb',
            'pfDeepCSVJetTags:probc',
            'pfDeepCSVJetTags:probudsg',
    ]
    ak4btaginfos = [ 'pfDeepCSVTagInfos' ] 

    jetToolbox(process, 'ak4', 'dummyseq', 'noOutput',
               dataTier='nanoAOD',
               PUMethod='Puppi', JETCorrPayload='AK4PFPuppi',
               #addQGTagger=True,
               runOnMC=True,
               Cut='pt > 15.0 && abs(eta) < 2.4',
               bTagDiscriminators=ak4btagdiscriminators,
               bTagInfos=ak4btaginfos,
               verbosity=4
               )

    #### AK8 PUPPI jets
    ak8btagdiscriminators = [
                        'pfBoostedDoubleSecondaryVertexAK8BJetTags',
                        'pfMassIndependentDeepDoubleBvLJetTags:probQCD',
                        'pfMassIndependentDeepDoubleBvLJetTags:probHbb',
                        'pfMassIndependentDeepDoubleCvLJetTags:probQCD',
                        'pfMassIndependentDeepDoubleCvLJetTags:probHcc',
                        'pfMassIndependentDeepDoubleCvBJetTags:probHbb',
                        'pfMassIndependentDeepDoubleCvBJetTags:probHcc',
            ]

    jetToolbox(process, 'ak8', 'adummyseq', 'noOutput',
               dataTier='nanoAOD',
               PUMethod='Puppi', JETCorrPayload='AK8PFPuppi',
               runOnMC=True,
               Cut='pt > 170.0 && abs(eta) < 2.4',
               bTagDiscriminators=ak8btagdiscriminators,
               addSoftDrop=True,
               addSoftDropSubjets=True,
               addPruning=True,
               addNsub=True,
               addEnergyCorrFunc=True,
               )
    return process

# ---------------------------------------------------------

def nanoJTB_customizeMC(process):
    run2_nanoAOD_94X2016.toModify(process, setupCustomizedJetToolbox)
    run2_nanoAOD_94XMiniAODv2.toModify(process, setupCustomizedJetToolbox)
    run2_nanoAOD_102Xv1.toModify(process, setupCustomizedJetToolbox)
    process.NANOAODSIMoutput.fakeNameForCrab = cms.untracked.bool(True)  # needed for crab publication
    return process

```
            
</p>
</details>

This file is already included in this tutorial. _Do not try to copy this file in your working directory._

Then to create a nanoAOD-like file, run the next line in your terminal, or in the next cell. 

In [ ]:
%%bash
cmsDriver.py nanoAOD_jetToolbox_cff -s NANO --mc --eventcontent NANOAODSIM --datatier NANOAODSIM -n 100  --conditions 102X_upgrade2018_realistic_v20 --era Run2_2018,run2_nanoAOD_102Xv1 --customise_commands="process.add_(cms.Service('InitRootHandlers', EnableIMT = cms.untracked.bool(False)))" --customise Analysis/JMEDAS/nanoAOD_jetToolbox_cff.nanoJTB_customizeMC --filein /store/mc/RunIIAutumn18MiniAOD/QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/102X_upgrade2018_realistic_v15-v1/210000/DA20DC21-E781-C540-9FCD-7BCF2144CA4E.root --fileout file:jetToolbox_nano_mc.root

The output file in this case, called `jetToolbox_nano_mc.root` will be a nanoAOD file with all the content from the central nanoAOD sample but including an extra collection which name starts with `selectedPatJetsAK8PFPuppi` (Or the name indicated in the jetToolbox printouts). 